Version: 02.14.2023

# Lab 2.1: Applying ML to an NLP Problem（实验室 2.1：将机器学习 (ML) 应用于 NLP 问题）

在本实验室内容中，您将使用 Amazon SageMaker 中的内置机器学习 (ML) 模型 __LinearLearner__ 来预测评论数据集的 __isPositive__ 字段。

## 业务场景简介
您在一家在线零售商店工作，该商店希望改进发布负面评论客户的客户参与度。该公司希望检测负面评论并将这些评论分配给客服人员来处理。

您的任务是使用机器学习 (ML) 检测负面评论来解决部分问题。您可以访问包含评论的数据集，这些评论已被归类为正面或负面评论。您将使用此数据集训练机器学习 (ML) 模型，以预测新评论的情绪。

## 关于该数据集
[AMAZON-REVIEW-DATA-CLASSIFICATION.csv](https://github.com/aws-samples/aws-machine-learning-university-accelerated-nlp/tree/master/data/examples) 文件包含实际的商品评论，这些评论包括文本数据和数字数据。每个评论都标记为 _positive (1)_ 或 _negative (0)_。

该数据集包含以下功能：
* __reviewText:__ 评论文本
* __summary:__ 评论总结
* __verified:__ 购买是否已验证（真还是假）
* __time:__ 评论的 UNIX 时间戳
* __log_votes:__ 对数调整的投票日志（1+ 票）
* __isPositive:__ 评论是正面还是负面的（1 还是 0）

本实验室的数据集是经 Amazon 许可提供给您的，并受 Amazon 权限和访问条款的约束（可在 https://www.amazon.com/gp/help/customer/display.html?nodeId=201909000 上获得）。明确禁止出于完成本课程之外的目的复制、修改、出售、导出或使用该数据集。

## 实验室步骤

要完成本实验室内容，您需要按以下步骤操作：

1. [读取数据集] (#1.-Reading-the-dataset)
2. [执行探索性数据分析] (#2.-Performing-exploratory-data-analysis)
3. [文本处理：删除非索引字以及提取词根](#3.-Text-processing:-Removing-stopwords-and-stemming)
4. [拆分训练、验证和测试数据](#4.-Splitting-training,-validation,-and-test-data)
5. [使用管道和列转换器处理数据](#5.-Processing-data-with-pipelines-and-a-ColumnTransformer)
6. [使用内置的 SageMaker 算法训练分类器](#6.-Training-a-classifier-with-a-built-in-SageMaker-algorithm)
7. [评估模型](#7.-Evaluating-the-model)
8. [将模型部署到端点](#8.-Deploying-the-model-to-an-endpoint)
9. [测试端点](#9.-Testing-the-endpoint)
10. [清理模型构件](#10.-Cleaning-up-model-artifacts)
    
## 提交作业

1.在实验室控制台中，选择 **Submit**（提交）记录您的进度，在出现提示时，选择 **Yes**（是）。

1.如果在几分钟后仍未显示结果，请返回到这些说明的顶部，并选择 **Grades**（成绩）。

     **提示**：您可以多次提交作业。您更改作业后，再次选择 **Submit**（提交）。您最后一次提交的作业将记为本实验室内容的作业。

1.要查找有关您作业的详细反馈，请选择 **Details**（详细信息），然后选择 **View Submission Report**（查看提交报告）。 

首先要安装/升级 pip、Sagemaker 以及 scikit-learn。

[scikit-learn](https://scikit-learn.org/stable/) 是一个开源机器学习库。它为模型拟合、数据预处理、模型选择和评估以及许多其他实用程序提供了各种工具。

In [ ]:
#Upgrade dependencies
!pip install --upgrade pip
!pip install --upgrade scikit-learn
!pip install --upgrade sagemaker
!pip install --upgrade botocore
!pip install --upgrade awscli

## 1.读取数据集
([回到页首](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

您将使用 __pandas__ 库读取数据集。[Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) 是用于数据分析的热门 python 库。它提供了数据操作、清理和数据整理功能以及可视化效果。

In [ ]:
import pandas as pd

df = pd.read_csv('../data/AMAZON-REVIEW-DATA-CLASSIFICATION.csv')

print('The shape of the dataset is:', df.shape)

查看数据集中的前五行。

In [ ]:
df.head(5)

您可以更改笔记本中的选项以显示更多的文本数据。

In [ ]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)
df.head()

如果需要，您可以查看具体的条目。

In [ ]:
print(df.loc[[580]])

清楚自己正在处理的数据类型是有益的。您可以在 DataFrame 上使用  `dtypes` 来显示类型。

In [ ]:
df.dtypes

## 2.执行探索性数据分析
([回到页首](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

现在，您将探究数据集的目标分布。

In [ ]:
df['isPositive'].value_counts()

业务问题关注的是找到负面评论 (_0_)。但是，线性学习器的模型优化默认为查找正值 (_1_)。通过切换负值 (_0_) 和正值 (_1_)，可以使此过程更加顺畅地进行。按此操作，您就可以更轻松地优化模型。

In [ ]:
df = df.replace({0:1, 1:0})
df['isPositive'].value_counts()

检查缺失值的数量：

In [ ]:
df.isna().sum()

文本字段会缺少值。通常情况下，您要决定如何处理这些缺失的值。您可以删除数据或填充以标准文本。

## 3.文本处理：删除非索引字以及提取词根
([回到页首](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

在此任务中，您将删除一些非索引字，然后对文本数据执行提取词根操作。您要对数据进行标准化，以减少必须处理的各种信息的量。

[nltk](https://www.nltk.org/) 是处理人类语言数据的热门平台。它提供了用于处理文本以执行分类、令牌化、提取词根、标记、解析和语义推理的接口和函数。

导入后，您只能下载所需的功能。在本示例中，您将使用：

– **punkt** 是句子分词器
– **stopwords** 提供了您可以使用的非索引字列表。

In [ ]:
# Install the library and functions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

您将在下一部分中创建删除非索引字和清理文本的流程。自然语言工具包 (NLTK) 库提供了常见非索引字列表。您将使用该列表，但是您首先要从该列表中删除一些单词。您在文本中保留的非索引字对于确定情绪很有用。

In [ ]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Get a list of stopwords from the NLTK library
stop = stopwords.words('english')

# These words are important for your problem. You don't want to remove them.
excluding = ['against', 'not', 'don', 'don\'t','ain', 'are', 'aren\'t', 'could', 'couldn\'t',
             'did', 'didn\'t', 'does', 'doesn\'t', 'had', 'hadn\'t', 'has', 'hasn\'t', 
             'have', 'haven\'t', 'is', 'isn\'t', 'might', 'mightn\'t', 'must', 'mustn\'t',
             'need', 'needn\'t','should', 'shouldn\'t', 'was', 'wasn\'t', 'were', 
             'weren\'t', 'won\'t', 'would', 'wouldn\'t']

# New stopword list
stopwords = [word for word in stop if word not in excluding]




Snowball 词干算法将提取词根。例如，“正在行走”将提取为“走”。

In [ ]:
snow = SnowballStemmer('english')

您必须对数据执行一些其他的标准化任务。以下函数将：

– 用空字符串替换所有缺失的值
– 将文本转换为小写
– 删除所有前导或长串空格
– 删除所有额外的空格和制表符
– 删除所有 HTML 标记

在  `for` 循环中，将保留并返回 __NOT__（非）数字、超过 2 个字符且不属于非索引字列表的所有单词。

In [ ]:
def process_text(texts): 
    final_text_list=[]
    for sent in texts:
        
        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ''
            
        filtered_sentence=[]
        
        sent = sent.lower() # Lowercase 
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:
        
        for w in word_tokenize(sent):
            # Applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stopwords
            if(not w.isnumeric()) and (len(w)>2) and (w not in stopwords):  
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) # Final string of cleaned words
 
        final_text_list.append(final_string)
        
    return final_text_list

## 4.拆分训练、验证和测试数据
([回到页首](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

在此步骤中，您将使用 sklearn [__train_test_split()__](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) 函数将数据集拆分为训练 (80％)、验证 (10％) 和测试 (10％)。

训练数据将用于训练模型，然后用测试数据进行测试。一旦模型训练完毕，就会使用验证集，以便为您提供模型对真实数据的处理效果的指标。

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df[['reviewText', 'summary', 'time', 'log_votes']],
                                                  df['isPositive'],
                                                  test_size=0.20,
                                                  shuffle=True,
                                                  random_state=324
                                                 )

X_val, X_test, y_val, y_test = train_test_split(X_val,
                                                y_val,
                                                test_size=0.5,
                                                shuffle=True,
                                                random_state=324)

通过数据集拆分，您现在可以对训练、测试和验证集中的每个文本要素运行上面定义的  `process_text` 函数。

In [ ]:
print('Processing the reviewText fields')
X_train['reviewText'] = process_text(X_train['reviewText'].tolist())
X_val['reviewText'] = process_text(X_val['reviewText'].tolist())
X_test['reviewText'] = process_text(X_test['reviewText'].tolist())

print('Processing the summary fields')
X_train['summary'] = process_text(X_train['summary'].tolist())
X_val['summary'] = process_text(X_val['summary'].tolist())
X_test['summary'] = process_text(X_test['summary'].tolist())

## 5.使用管道和列转换器处理数据
([回到页首](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

在使用数据训练模型之前，您通常会对数据执行许多任务。还必须对模型部署后用于推断的所有数据执行这些步骤。一种组织这些步骤的好方法是定义 _pipeline_。管道是将对数据执行的处理任务的集合。可以创建不同的管道来处理不同的字段。由于您同时使用文本和数字数据，因此可以定义以下管道：

   * 对于数值要素管道，__numerical_processor__ 使用 MinMaxScaler。（在使用决策树时，不需要缩放要素，但最好了解如何使用更多的数据转换。） 如果要对不同的数字要素执行不同类型的处理，则应构建不同的管道，例如为两个文本要素显示的管道。
   * 对于文本要素管道，__text_processor__ 为文本字段使用  `CountVectorizer()`。
   
然后，对数据集要素的选择性准备工作将整合到一个集合性列转换器中，该转换器将在管道中与估算器一同使用。此过程可确保在拟合模型或进行预测时，对原始数据自动执行转换。（例如，当您通过交叉验证在验证数据集上评估模型时，或者将来对测试数据集进行预测时。）

In [ ]:
# Grab model features/inputs and target/output
numerical_features = ['time',
                      'log_votes']

text_features = ['summary',
                 'reviewText']

model_features = numerical_features + text_features
model_target = 'isPositive'

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

### COLUMN_TRANSFORMER ###
##########################

# Preprocess the numerical features
numerical_processor = Pipeline([
    ('num_imputer', SimpleImputer(strategy='mean')),
    ('num_scaler', MinMaxScaler()) 
                                ])
# Preprocess 1st text feature
text_processor_0 = Pipeline([
    ('text_vect_0', CountVectorizer(binary=True, max_features=50))
                                ])

# Preprocess 2nd text feature (larger vocabulary)
text_precessor_1 = Pipeline([
    ('text_vect_1', CountVectorizer(binary=True, max_features=150))
                                ])

# Combine all data preprocessors from above (add more, if you choose to define more!)
# For each processor/step specify: a name, the actual process, and finally the features to be processed
data_preprocessor = ColumnTransformer([
    ('numerical_pre', numerical_processor, numerical_features),
    ('text_pre_0', text_processor_0, text_features[0]),
    ('text_pre_1', text_precessor_1, text_features[1])
                                    ]) 

### DATA PREPROCESSING ###
##########################

print('Datasets shapes before processing: ', X_train.shape, X_val.shape, X_test.shape)

X_train = data_preprocessor.fit_transform(X_train).toarray()
X_val = data_preprocessor.transform(X_val).toarray()
X_test = data_preprocessor.transform(X_test).toarray()

print('Datasets shapes after processing: ', X_train.shape, X_val.shape, X_test.shape)

请注意数据集中的要素数量如何从 4 个增加到 202 个。

In [ ]:
print(X_train[0])

## 6.使用内置的 SageMaker 算法训练分类器
([回到页首](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

在此步骤中，您将使用以下选项调用 Sagemaker `LinearLearner()` 算法：
* __Permissions -__ `role` 设置为当前环境中的 AWS Identity and Access Management (IAM) 角色。
* __Compute power -__ 您将使用  `train_instance_count` 参数和  `train_instance_type` 参数。本示例使用  `ml.m4.xlarge` 资源进行训练。您可以根据需要更改实例类型。（例如，您可以将 GPU 用于神经网络。） 
* __Model type -__ `predictor_type` 设置为 __`binary_classer`__，因为您正在处理二元分类问题。如果涉及三个或更多类，您可以使用 __`multiclass_classifier`__，也可以使用 __`regressor`__ 来解决回归问题。


In [ ]:
import sagemaker

# Call the LinearLearner estimator object
linear_classifier = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                           instance_count=1,
                                           instance_type='ml.m4.xlarge',
                                           predictor_type='binary_classifier')

要设置估算器的训练、验证和测试部分，您可以使用  `binary_estimator` 的函数  `record_set()`。

In [ ]:
train_records = linear_classifier.record_set(X_train.astype('float32'),
                                            y_train.values.astype('float32'),
                                            channel='train')
val_records = linear_classifier.record_set(X_val.astype('float32'),
                                          y_val.values.astype('float32'),
                                          channel='validation')
test_records = linear_classifier.record_set(X_test.astype('float32'),
                                           y_test.values.astype('float32'),
                                           channel='test')

 `fit()` 函数应用随机梯度下降 (SGD) 算法的分布式版本，而且您要将数据发送给它。 `logs=False` 设置禁用了这些日志。您可以删除该参数以查看有关该过程的更多详细信息。__在 ml.m4.xlarge 实例上，此过程大约需要 3-4 分钟。__

In [ ]:
linear_classifier.fit([train_records,
                       val_records,
                       test_records],
                       logs=False)

## 7.评估模型
([回到页首](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

您可以使用 SageMaker 分析在测试集上获取（自己选择的）一些绩效指标。这个过程不需要您部署模型。

线性学习器提供训练期间计算的指标。您可以在优化模型时使用这些指标。验证集的可用指标包括：

- objective_loss - 对于二元分类问题，这将是每个纪元物流损失的平均值
- binary_classification_accuracy - 数据集中最终模型的准确性，即模型进行了多少正确预测
- precision - 量化确实是正面的正面类别预测数量
- recall - 量化正面类别预测的数量
- binary_f_beta - 精度和召回指标的调和平均值

在此示例中，您感兴趣的是有多少预测是正确的。使用 **binary_classification_accuracy** 指标似乎是合适的。

In [ ]:
sagemaker.analytics.TrainingJobAnalytics(linear_classifier._current_job_name, 
                                         metric_names = ['test:binary_classification_accuracy']
                                        ).dataframe()

您应当看到一个约为 0.85 的值。您的值可能会有所不同，但应该接近此值。这意味着模型在 85％ 时间里准确地预测了正确答案。根据立项分析，您可能需要使用超参数优化任务进一步优化模型，或者进行更多的特征工程。

## 8.将模型部署到端点
([回到页首](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

在本练习的最后一部分中，您要将模型部署到自己选择的另一个实例。您可以在生产环境中使用此模型。已部署的端点可以与其他 AWS 服务结合使用，例如 AWS Lambda 和 Amazon API Gateway。如果您有兴趣了解更多信息，请参阅以下演练：[使用 Amazon API Gateway 和 AWS Lambda 调用 Amazon SageMaker 模型端点] (https://aws.amazon.com/blogs/machine-learning/call-an-amazon-sagemaker-model-endpoint-using-amazon-api-gateway-and-aws-lambda/)。

要部署模型，请运行以下单元。您可以使用不同的实例类型，例如 _ml.t2.medium_、_ml.c4.xlarge_ 等。__这个过程需要一些时间才能完成（大约 7-8 分钟）。__

In [ ]:

linear_classifier_predictor = linear_classifier.deploy(initial_instance_count = 1,
                                                       instance_type = 'ml.c5.large'
                                                      )

## 9.测试端点
([回到页首](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

现在已部署端点，您将向其发送测试数据并从数据中获取预测。

In [ ]:
import numpy as np

# Get test data in batch size of 25 and make predictions.
prediction_batches = [linear_classifier_predictor.predict(batch)
                      for batch in np.array_split(X_test.astype('float32'), 25)
                     ]

# Get a list of predictions
print([pred.label['score'].float32_tensor.values[0] for pred in prediction_batches[0]])

## 10.清理模型构件
([回到页首](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

使用完端点后，可以运行以下命令删除端点。

**提示：** - 请记住，在使用自己的账户时，如果不删除端点和其他资源，您将产生费用。

In [ ]:
linear_classifier_predictor.delete_endpoint()

# 恭喜！

在本实验室内容中，您分析了一个非常简单的 NLP 问题。使用带标签的数据集，您使用简单的分词器和编码器来生成训练线性学习器模型所需的数据。然后，您部署了模型并执行了一些预测。如果您亲手执行这些任务，您可能需要获取数据并予以标记以进行训练。另一种方法可能是使用预训练的算法或托管的服务。您也可能要使用超参数优化任务进一步优化模型。

您已经完成了本实验室内容，现在可以按照实验室指南中的说明结束本实验室内容。

©2023 Amazon Web Services, Inc. 或其联属公司。保留所有权利。未经 Amazon Web Services, Inc. 事先书面许可，不得复制或转载本文的部分或全部内容。禁止因商业目的复制、出借或出售本文。所有商标均为各自所有者的财产。*